# Modelling for predicting the spring onset of natural vegetation across the Northern Hemisphere

https://doi.org/10.1016/j.ecolind.2021.108126


## Data

- Observation data from the USA National Phenology Network (USA-NPN)
- MODIS land cover type product and the land cover dynamics product
- Daymet gridded climate data covering Northern America
- Climate data obtained from Global Land Data Assimilation System (GLDAS) covering the Northern Hemisphere

### USA-NPN

- Observation records of vegetation phenology from 2895 sites from 2007 to 2017
- Four vegetation types defined in the USA-NPN dataset, including deciduous broadleaf forests (DBF), evergreen broadleaf forests (EBF), graminoid (GRA), and drought deciduous broadleaf forests (DDBF)
- Spring onset: breaking leaf buds for DBF, breaking leaf buds for ENF, breaking leaf buds for DDBF, and initial growth of grasses/sedges for GRA
- Averaging the timing of spring onsets for all records at the same site in the same year
- Excluding observation data in the sites located in the pixels that are classified as urban and build-up in the MODIS land cover maps

In [1]:
import springtime
from springtime.datasets.rnpn import RNPN, npn_species, npn_phenophases

# Find species

In [2]:
# List IDs and names for available species, phenophases
species = npn_species()
species.head()

,species_id,common_name,genus,genus_id,genus_common_name,species,kingdom,itis_taxonomic_sn,functional_type,class_id,class_common_name,class_name,order_id,order_common_name,order_name,family_id,family_name,family_common_name
1,120,'ohi'a lehua,Metrosideros,798,Lehuas (Metrosideros),polymorpha,Plantae,27259,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,89,Myrtle and Evening-primrose Families,Myrtales,301,Myrtaceae,Myrtle Family
2,1436,absinthium,Artemisia,437,Sagebrushes (Artemisia),absinthium,Plantae,35445,Forb,15,Flowering Plants,Magnoliopsida,69,"Aster, Bellflower and Buckbean Families",Asterales,242,Asteraceae,Aster Family
3,1227,Acadian flycatcher,Empidonax,612,Empidonax Flycatchers (Empidonax),virescens,Animalia,178339,Bird,5,Birds,Aves,31,Perching Birds,Passeriformes,154,Tyrannidae,Tyrant Flycatchers
4,1229,acorn woodpecker,Melanerpes,790,Melanerpine Woodpeckers (Melanerpes),formicivorus,Animalia,178189,Bird,5,Birds,Aves,33,Woodpeckers,Piciformes,158,Picidae,Woodpeckers
5,2110,Adam and Eve,Aplectrum,1285,Adam and Eves (Aplectrum),hyemale,Plantae,43489,Forb,15,Flowering Plants,Magnoliopsida,68,"Asparagas, Iris, Orchid and Aloe Families",Asparagales,307,Orchidaceae,Orchid Family


In [25]:
from springtime.utils import NamedIdentifiers

def npn_species_ids_by_functional_type(functional_type):
    species = lookup(npn_species(), "functional_type", functional_type)
    return NamedIdentifiers(name=functional_type, items=species.species_id.to_list())

def npn_phenophase_ids_by_name(phenophase_name):
    phenophases = lookup(npn_phenophases(), "phenophase_name", phenophase_name)
    return NamedIdentifiers(name=phenophase_name, items=phenophases.phenophase_id.to_list())

def lookup(df, column, expression):
    """Return rows where column matches expression."""
    return df[df[column].str.lower().str.contains(expression.lower())]

In [16]:
deciduous_broadleaf = npn_species_ids_by_functional_type("Deciduous broadleaf")
deciduous_broadleaf

NamedIdentifiers(name='Deciduous broadleaf', items=[1174, 1446, 93, 812, 79, 1832, 724, 1048, 71, 823, 91, 206, 1762, 970, 1455, 76, 1245, 1246, 810, 126, 444, 1208, 1362, 1007, 1353, 20, 717, 1447, 2087, 1173, 1857, 124, 320, 1483, 1461, 982, 72, 1485, 1878, 1875, 1880, 1355, 105, 802, 1, 1188, 778, 87, 1176, 873, 28, 1849, 1481, 90, 839, 1458, 33, 1591, 1159, 1013, 80, 1494, 440, 941, 1510, 1691, 1015, 122, 2093, 2046, 328, 2, 799, 859, 1462, 2042, 101, 1452, 788, 2131, 1466, 2143, 765, 858, 713, 1211, 1161, 1632, 64, 1373, 1181, 1649, 871, 1682, 776, 1358, 2085, 1049, 2066, 2052, 1212, 1748, 1045, 2050, 29, 293, 796, 1807, 89, 201, 829, 36, 1364, 303, 2101, 1431, 92, 897, 1499, 125, 2145, 1759, 108, 896, 1489, 1732, 828, 1341, 22, 832, 1071, 835, 944, 322, 1493, 430, 2053, 1000, 1817, 804, 976, 1733, 708, 1044, 7, 1780, 854, 1432, 1736, 1781, 1760, 1363, 1771, 836, 1913, 1006, 2142, 12, 73, 1066, 1367, 1843, 977, 2027, 988, 1871, 996, 1744, 1163, 1636, 1470, 62, 1805, 1448, 319, 75,

In [22]:
evergreen_broadleaf = npn_species_ids_by_functional_type("Evergreen broadleaf")
evergreen_broadleaf

NamedIdentifiers(name='Evergreen broadleaf', items=[120, 1343, 1861, 1847, 1717, 1982, 1365, 1804, 1068, 857, 106, 757, 1178, 1910, 313, 905, 790, 1968, 1803, 706, 1444, 111, 298, 1004, 1160, 1057, 1166, 705, 877, 1970, 925, 2057, 2030, 1887, 701, 981, 1206, 218, 1067, 1346, 2080, 702, 1055, 119, 117, 1003, 2045, 1593, 441, 2164, 2064, 879, 112, 987, 1690, 895, 887, 880, 1504, 1938, 933, 221, 2139, 435, 1976, 1799, 913, 1490, 761, 1954, 974, 1845, 115, 1002, 980, 924, 113, 2054, 1247, 769, 442, 104, 103, 121, 301, 318, 1985, 1870, 324, 785, 995, 1975, 329, 305, 1961, 226, 907, 1644, 986, 443, 1688, 1609, 904, 1927, 760, 1430, 860, 1475, 109, 1360, 114, 793, 716, 1594, 1322, 2006, 743, 1628, 1324, 1689, 1469, 1443, 110, 1824, 127, 1969, 1479, 1681, 1830, 1359, 311, 830, 297, 306, 1892, 985, 903, 2092, 979, 1058, 327, 107, 1170, 1069, 1474, 721, 1720, 1368, 1626, 909, 1323, 1934, 707, 861, 1349, 2051, 878, 722, 1602, 1922, 116, 1011, 1846, 766, 222, 1351, 1467, 1766])

# Phenophases

In [5]:
phenophases = npn_phenophases()
phenophases.head()

,phenophase_id,phenophase_name,phenophase_category,color,pheno_class_id
1,56,First leaf,Leaves,NA_character_,1
2,57,75% leaf elongation,Leaves,NA_character_,2
3,58,First flower,Flowers,NA_character_,7
4,59,Last flower,Flowers,NA_character_,9
5,60,First fruit ripe,Fruits,NA_character_,12


In [26]:
breaking_leaf_buds = npn_phenophase_ids_by_name("Breaking leaf buds")
breaking_leaf_buds

NamedIdentifiers(name='Breaking leaf buds', items=[371, 373])

In [27]:
initial_growth = npn_phenophase_ids_by_name("initial growth")
initial_growth

NamedIdentifiers(name='initial growth', items=[482, 492, 508])

### Data preparation

In [28]:
npn_broadleaf = RNPN(years=(2007, 2018), species_ids=deciduous_broadleaf, phenophase_ids=breaking_leaf_buds)
npn_broadleaf

RNPN(dataset='RNPN', years=(2007, 2018), species_ids=NamedIdentifiers(name='Deciduous broadleaf', items=[1174, 1446, 93, 812, 79, 1832, 724, 1048, 71, 823, 91, 206, 1762, 970, 1455, 76, 1245, 1246, 810, 126, 444, 1208, 1362, 1007, 1353, 20, 717, 1447, 2087, 1173, 1857, 124, 320, 1483, 1461, 982, 72, 1485, 1878, 1875, 1880, 1355, 105, 802, 1, 1188, 778, 87, 1176, 873, 28, 1849, 1481, 90, 839, 1458, 33, 1591, 1159, 1013, 80, 1494, 440, 941, 1510, 1691, 1015, 122, 2093, 2046, 328, 2, 799, 859, 1462, 2042, 101, 1452, 788, 2131, 1466, 2143, 765, 858, 713, 1211, 1161, 1632, 64, 1373, 1181, 1649, 871, 1682, 776, 1358, 2085, 1049, 2066, 2052, 1212, 1748, 1045, 2050, 29, 293, 796, 1807, 89, 201, 829, 36, 1364, 303, 2101, 1431, 92, 897, 1499, 125, 2145, 1759, 108, 896, 1489, 1732, 828, 1341, 22, 832, 1071, 835, 944, 322, 1493, 430, 2053, 1000, 1817, 804, 976, 1733, 708, 1044, 7, 1780, 854, 1432, 1736, 1781, 1760, 1363, 1771, 836, 1913, 1006, 2142, 12, 73, 1066, 1367, 1843, 977, 2027, 988, 1871, 

In [29]:
npn_broadleaf.download()

downloading /tmp/data/rnpn/rnpn_npn_data_y_2007_2018_Deciduous broadleaf_Breaking leaf buds.csv


R[write to console]: using a custom handler function.

R[write to console]: opening curl input connection.



 Found 13 records...

R[write to console]: closing curl input connection.

R[write to console]: using a custom handler function.

R[write to console]: opening curl input connection.



 Found 28 records...

...

 Found 1209 records...

R[write to console]: closing curl input connection.

R[write to console]: using a custom handler function.

R[write to console]: opening curl input connection.



 Found 1340 records...

R[write to console]: closing curl input connection.

R[write to console]: using a custom handler function.

R[write to console]: opening curl input connection.

R[write to console]: 

R[write to console]: closing curl input connection.



RRuntimeError: 